In [7]:
# Test - Merge OCR and VC Drafted Decks
# Compares OCR-validated deck lists against VC drafted_decks data.
# Requires: pip install pandas requests

import re, glob, os, time
import pandas as pd
import requests

BASE_DIR       = os.path.abspath(os.path.join(os.getcwd(), '..'))
DRAFT_DATA_DIR = os.path.join(BASE_DIR, 'Draft csv data')
CARDLIST_DIR   = os.path.join(BASE_DIR, 'data', 'cardlist')
OUTPUT_DIR     = os.path.join(os.getcwd(), 'output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ── Load cube list (name ↔ scryfall_id lookup) ──
cube_lists = sorted(
    glob.glob(os.path.join(CARDLIST_DIR, 'dimlas*_cardlist.csv')),
    key=lambda f: int(re.search(r'dimlas(\d+)_cardlist', f).group(1)),
    reverse=True
)
if not cube_lists:
    raise FileNotFoundError(f'No dimlas*_cardlist.csv found in {CARDLIST_DIR}')

df_cube = pd.read_csv(cube_lists[0]).dropna(subset=['scryfall_id'])
id_to_name = dict(zip(df_cube['scryfall_id'], df_cube['name'].str.strip()))
name_to_id = {v: k for k, v in id_to_name.items()}

total = len(pd.read_csv(cube_lists[0]))
found = len(id_to_name)
print(f'Cube list: {os.path.basename(cube_lists[0])} — {found}/{total} cards with scryfall ID')
if found == total:
    print('✓ All cards have a scryfall ID.')

Cube list: dimlas5_cardlist.csv — 540/540 cards with scryfall ID
✓ All cards have a scryfall ID.


In [8]:
# ── Load both sources ────────────────────────────────────────────────────────

# 1) OCR: clean_*.csv from newest draft folder in data/clean/
CLEAN_DIR = os.path.join(BASE_DIR, 'data', 'clean')
clean_draft_folders = sorted(
    [d for d in glob.glob(os.path.join(CLEAN_DIR, '*')) if os.path.isdir(d)],
    reverse=True
)
if not clean_draft_folders:
    raise FileNotFoundError(f'No draft folders found in {CLEAN_DIR}')
newest_clean = clean_draft_folders[0]
print(f'OCR folder: {os.path.basename(newest_clean)}')

ocr_frames = []
for f in sorted(glob.glob(os.path.join(newest_clean, 'clean_*.csv'))):
    player = os.path.basename(f).replace('clean_', '').replace('.csv', '')
    df = pd.read_csv(f)
    df.columns = df.columns.str.strip().str.lower()
    df['player'] = player
    ocr_frames.append(df[['player', 'name']])

if not ocr_frames:
    raise FileNotFoundError(f'No clean_*.csv files found in {newest_clean}')

df_ocr = pd.concat(ocr_frames, ignore_index=True)
df_ocr.columns = ['player', 'card_name']
df_ocr['card_name'] = df_ocr['card_name'].str.strip()
print(f'  {len(df_ocr)} cards from {df_ocr["player"].nunique()} players')

# 2) VC: newest drafted_decks.csv → resolve scryfallId to card name
drafted = sorted(glob.glob(os.path.join(DRAFT_DATA_DIR, '*_drafted_decks.csv')), reverse=True)
if not drafted:
    raise FileNotFoundError('No *_drafted_decks.csv found')
print(f'\nVC file: {os.path.basename(drafted[0])}')

df_vc = pd.read_csv(drafted[0])[['player', 'scryfallId']].copy()
df_vc.columns = ['player', 'scryfall_id']
df_vc['player']    = df_vc['player'].str.strip()
df_vc['card_name'] = df_vc['scryfall_id'].map(id_to_name)

# Batch-fetch any IDs not in the cube list (old printings / swapped cards)
missing_ids = df_vc.loc[df_vc['card_name'].isna(), 'scryfall_id'].unique().tolist()
if missing_ids:
    print(f'  Fetching {len(missing_ids)} IDs not in cube list from Scryfall...')
    api_map = {}
    for i in range(0, len(missing_ids), 75):
        batch = missing_ids[i:i+75]
        resp = requests.post('https://api.scryfall.com/cards/collection',
                             json={'identifiers': [{'id': s} for s in batch]}, timeout=30)
        resp.raise_for_status()
        for card in resp.json().get('data', []):
            api_map[card['id']] = card['name'].split(' // ')[0].strip()
        time.sleep(0.1)
    df_vc['card_name'] = df_vc.apply(
        lambda r: api_map.get(r['scryfall_id'], r['card_name']) if pd.isna(r['card_name']) else r['card_name'], axis=1)
    print(f'  Resolved {len(api_map)}/{len(missing_ids)} via API')

df_vc = df_vc[['player', 'card_name', 'scryfall_id']].dropna(subset=['card_name'])
print(f'  {len(df_vc)} cards from {df_vc["player"].nunique()} players')

OCR folder: 20260125_Draft_7
  346 cards from 12 players

VC file: 2026_01_25_drafted_decks.csv
  345 cards from 12 players


In [9]:
# ── Compare OCR vs VC ────────────────────────────────────────────────────────

# Normalize player names (strip surrounding whitespace only)
normalize = lambda s: str(s).strip()
df_ocr['player'] = df_ocr['player'].map(normalize)
df_vc['player']  = df_vc['player'].map(normalize)

# Merge on player + card_name
df_ocr['in_ocr'] = True
df_vc['in_vc']   = True

df_merged = pd.merge(df_ocr, df_vc, on=['player', 'card_name'], how='outer')
df_merged['in_ocr'] = df_merged['in_ocr'].fillna(False)
df_merged['in_vc']  = df_merged['in_vc'].fillna(False)

# Fill scryfall_id for OCR-only rows via name lookup
df_merged['scryfall_id'] = df_merged['scryfall_id'].fillna(
    df_merged['card_name'].map(name_to_id))

df_merged = df_merged.sort_values(['player', 'card_name']).reset_index(drop=True)

df_merged.to_csv(os.path.join(OUTPUT_DIR, 'combined_player_cards.csv'), index=False)

# Report
ocr_only = df_merged[df_merged['in_ocr'] & ~df_merged['in_vc']]
vc_only  = df_merged[~df_merged['in_ocr'] &  df_merged['in_vc']]

print(f'In both         : {(df_merged["in_ocr"] & df_merged["in_vc"]).sum()}')
print(f'OCR only        : {len(ocr_only)}')
print(f'VC only         : {len(vc_only)}')

if len(ocr_only) > 0:
    print('\n── In OCR but not in VC ──')
    display(ocr_only[['player', 'card_name']].reset_index(drop=True))

if len(vc_only) > 0:
    print('\n── In VC but not in OCR ──')
    display(vc_only[['player', 'card_name']].reset_index(drop=True))

assert len(ocr_only) == 0, f"FAIL: {len(ocr_only)} card(s) in OCR but not in VC"
assert len(vc_only)  == 0, f"FAIL: {len(vc_only)} card(s) in VC but not in OCR"
print('\n✓ Perfect match — both sources agree on every card for every player.')

In both         : 345
OCR only        : 346
VC only         : 0

── In OCR but not in VC ──


,player,card_name
0,Andrin,Ancestral Recall
1,Andrin,Bloodchief's Thirst
2,Andrin,Brazen Borrower
3,Andrin,Counterspell
4,Andrin,Cryptic Coat
...,...,...
341,Yannik,"Titania, Protector of Argoth"
342,Yannik,Underground Mortuary
343,Yannik,Unstoppable Slasher
344,Yannik,Vaultborn Tyrant


AssertionError: FAIL: 346 card(s) in OCR but not in VC